# Interfacing ichor files with other Computational Chemistry Python packages

The ichor file system can be readily interfaced with other parsers for file reading. Below is an example of using ichor along with the ase Python package for reading and writing a Gaussian input file.

### Interfacing with Atomic Simulation Environment, ASE (https://wiki.fysik.dtu.dk/ase/)

In [22]:
# ASE code for writing out file

from ase.io.gaussian import write_gaussian_in
from ase.atoms import Atoms as ASEAtoms
from ase.atoms import Atom as ASEAtom

ase_atoms_to_write = ASEAtoms([ASEAtom("N", (1.30610788, -29.77550072, -0.39451506)), ASEAtom("H", (0.88322943, -29.08071028, -1.14190493)),
              ASEAtom("H", (1.46749713, -29.22282070, 0.46703669)),
              ASEAtom("H", (2.11921902, -30.18852549, -0.75438182))])

# write out file to disk
# with open("../../../example_files/ase_example.gjf", "w") as f:

#     write_gaussian_in(f, ase_atoms_to_write, method="b3lyp", basis="aug-cc-pvtz", mult=1, charge=0)

In [23]:
from pathlib import Path
from typing import List, Optional, Union
from ichor.core.common.functools import classproperty
from ichor.core.files.file import FileContents, ReadFile, WriteFile
from ase.io.gaussian import read_gaussian_in, write_gaussian_in
from ichor.core.atoms import Atoms, Atom
from ichor.core.files.file_data import HasAtoms


class GJF(ReadFile, WriteFile, HasAtoms):
    """
    Implementation for interfacing with ASE
    """

    def __init__(
        self,
        path: Union[Path, str],
        charge: Optional[int] = None,
        spin_multiplicity: Optional[int] = None,
        ase_atoms: Optional[ASEAtoms] = None,
        basis_set: Optional[str] = None,
        method: Optional[str] = None
    ):
        super().__init__(path)

        self.charge: int = charge or FileContents
        self.spin_multiplicity: int = spin_multiplicity or FileContents
        self.ase_atoms = ase_atoms or FileContents
        self.atoms = FileContents
        self.basis_set = basis_set or FileContents
        self.method = method or FileContents

    @classproperty
    def filetype(self) -> str:
        return ".gjf"

    def _read_file(self):

        with open(self.path, "r") as f:
            all_lines = f.readlines()

        ase_atoms = read_gaussian_in(all_lines, attach_calculator=True)

        self.ase_atoms = ase_atoms
        self.charge = self.charge or self.ase_atoms.calc.parameters.get("charge")
        self.spin_multiplicity = self.spin_multiplicity or self.ase_atoms.calc.parameters.get("mult")
        self.basis_set = self.ase_atoms.calc.parameters.get("basis")
        self.method = self.ase_atoms.calc.parameters.get("method")


        ichor_atoms = Atoms()
        for a in self.ase_atoms:
            ichor_atoms.append(Atom(a.symbol, *a.position))

        self.atoms = ichor_atoms

    def _write_file(self, path: Path, *args, **kwargs):

        with open(path, "w") as f:

            write_gaussian_in(f, self.ase_atoms, basis_set=self.basis_set, method=self.method)


In [24]:
gjf_file_path = Path("../../../example_files/ase_example.gjf")

gjf_instance = GJF(gjf_file_path)

print(gjf_instance.method)
print(gjf_instance.basis_set)
print(gjf_instance.spin_multiplicity)
print(gjf_instance.charge)

b3lyp
aug-cc-pvtz
1
0


In [25]:
# case where we want to modify the GJF file to use a different basis set
gjf_file_path = Path("../../../example_files/ase_example.gjf")
gjf_instance = GJF(gjf_file_path, basis_set="6-31G(d,p)")
print(gjf_instance.basis_set)
# as it can be seen, basis set in the instance has changed.
# We need to overwrite the original file to make the basis set change for the calculation.
#gjf_instance.write()

6-31G(d,p)


### Interfacing with cclib (https://cclib.github.io/)

The same process can also be implemented for interfacing with cclib parsers. CClib is focused on parsing calculation output files only.

In [26]:
import cclib
from ichor.core.common.constants import type2nuclear_charge

charge_to_type = {int(v):k for k, v in type2nuclear_charge.items()}

class GaussianOutput(ReadFile, HasAtoms):
    """
    Example of Gaussian output file reading with ccib
    """

    def __init__(self, path):

        super().__init__(path)
        self.global_forces = FileContents
        self.charge = FileContents
        self.multiplicity = FileContents
        self.moments = FileContents
        self.atoms = FileContents

    def _read_file(self):

        data = cclib.io.ccread(filename)

        self.charge = data.charge
        self.multiplicity = data.mult
        self.global_forces = data.grads
        self.moments = data.moments

        # index because it has 1 more dimension that needed
        coords = data.atomcoords[0]
        atomic_numbers = data.atomnos

        atoms = Atoms()
        for num, coord in zip(atomic_numbers, coords):
            atoms.append(Atom(charge_to_type[num], *coord))

        self.atoms = atoms


# read in Gaussian output file
filename = "../../../example_files/urea_example_points_directory/urea0000/10k_random_sample_set_5000K05000.gau"
gaussian_out = GaussianOutput(filename)
print(gaussian_out.atoms)

C1      -2.93300000      0.92700000      0.52200000
O2      -3.41000000      0.38500000      1.39400000
N3      -2.28800000      2.07200000      0.74700000
N4      -3.10900000      0.28400000     -0.80000000
H5      -3.78100000     -0.22800000     -0.92900000
H6      -2.69400000      0.43100000     -1.68300000
H7      -2.31600000      2.68800000      1.50900000
H8      -1.56100000      1.99100000     -0.18800000


In the future, it is likely that other parsing libraries will be used by ichor to parse input and output files, as parsers already exist for most files. However, the documentation covers use cases for when there are no existing parsers available and ichor users must implement their own parsers. 

**There are still multiple benefits of using ichor because lazy file reading is inherently present, even if third party parsers are used to parse input and output files. In addition, ichor allows a more convenient interface where one single Python class allows users to read, write, and modify files through one single Python class. In addition, users can then easily submit and organize jobs on HPC clusters through the hpc package.**